In [31]:
!pip install google-cloud-texttospeech
!pip install SpeechRecognition
!pip install nltk


In [32]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
os.environ['DNNL_ENGINE_LIMIT_CPU_CAPABILITIES'] = '1'
os.environ['DNNL_VERBOSE'] ='1'
os.environ['SYCL_DEVICE_FILTER'] = 'opencl:gpu'

In [33]:
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
import os
import json
from google.cloud import texttospeech_v1
import nltk
nltk.download('punkt')
import random
import speech_recognition as sr
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from difflib import SequenceMatcher

# Set the path to your JSON key file
json_key_path = "/content/prefab-wave-395613-26155198bb68.json"

# Set up Google Cloud Text-to-Speech client using the service account key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_key_path
client = texttospeech_v1.TextToSpeechClient()

# Load intents from intents.json
intents_file_path = '/content/intents.json'
with open(intents_file_path, 'r') as file:
    intents = json.load(file)

print("Loaded intents:", intents)

# Initialize stemmer
stemmer = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

def get_matching_tag(input_tokens):
    tag_keywords = [intent['tag'] for intent in intents['intents']]
    for token in input_tokens:
        for keyword in tag_keywords:
            if token.lower() in keyword.lower():
                return keyword
    return None

def text_to_speech(text, output_file):
    synthesis_input = texttospeech_v1.SynthesisInput(text=text)
    voice_params = texttospeech_v1.VoiceSelectionParams(
        language_code="en-US", ssml_gender=texttospeech_v1.SsmlVoiceGender.NEUTRAL
    )
    audio_config = texttospeech_v1.AudioConfig(
        audio_encoding=texttospeech_v1.AudioEncoding.MP3
    )
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice_params, audio_config=audio_config
    )
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
    print(f"Audio content written to {output_file}")

def generate_response(tag):
    for intent in intents['intents']:  # Access 'intents' key
        if intent['tag'].startswith(tag):  # Check if tag starts with the matched keyword
            responses = intent['responses']
            return random.choice(responses)
    return "Please be more specific."

# Initialize the recognizer
recognizer = sr.Recognizer()

# Input WAV file
input_wav_path = input("Enter the path to the input WAV file: ")

# Process the WAV file
with sr.AudioFile(input_wav_path) as source:
    audio = recognizer.record(source)
    print("Processing the WAV file...")

    try:
        input_text = recognizer.recognize_google(audio)
        print("You said:", input_text)

        preprocessed_input = preprocess_text(input_text)
        print("Preprocessed input:", preprocessed_input)

        matched_tag = get_matching_tag(preprocessed_input)
        if matched_tag:
            response = generate_response(matched_tag)
            print("Response:", response)

            # Save response as MP3
            output_directory = input("Enter the directory path to save the output MP3 file: ")
            while not os.path.isdir(output_directory):
                output_directory = input("Please enter a valid directory path to save the output MP3 file: ")

            output_mp3_path = os.path.join(output_directory, "response.wav")
            text_to_speech(response, output_mp3_path)  # Use the text_to_speech function
            print("Output MP3 saved at:", output_mp3_path)
        else:
            print("No matching tag found.")

    except sr.UnknownValueError:
        print("Could not understand audio")
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loaded intents: {'intents': [{'tag': 'Cuts', 'patterns': ['What to do if Cuts?', 'How to cure Cuts?', 'Which medicine to apply for Cuts?', 'what to apply on cuts?', 'Cuts'], 'responses': ['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.'], 'context_set': ''}, {'tag': 'Abrasions', 'patterns': ['how do you treat abrasions?', 'Do Abrasions cause scars?', 'Abrasions', 'what to do if abrasions?', 'Which medicine to apply for abrasions?', 'How to cure abrasions?'], 'responses': ['Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth